<a href="https://colab.research.google.com/github/Gabor132/QuarantineSportsClassifier/blob/master/src/QuarantineSports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quarantine Sports Classifier 

## Imports

In [1]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


## Constants

In [0]:
learning_rate = 0.1
testing_data_percentage = 0.2
nr_of_frames_per_sequence = 15
nr_of_keypoints = 25
nr_of_values_per_keypoint = 3
dataset_path = "/content/drive/My Drive/QuarantineSportsDatasets/Dataset/OpenPoseDataset/video2.json"

## Load Dataset and create X and Y

In [0]:
df = pd.read_json(dataset_path)

## Important to know

1.   Out of open pose we extract for all frames all the 25 keypoints (each keypoint contains the X, Y and score)
2.   The Category exists per frame and is 0/1 as in Wrong/Correct as in Not a Push-Up Frame/Is a Push Up Frame


In [0]:

keypoints = df['Keypoints'].values
y_total = df['Category'].values


xList = []
for k in keypoints:
    if k != None:
        newK = np.reshape(np.asarray(k), (25,3))
        xList.append(newK)
    else:
        xList.append(np.reshape(np.zeros(75), (25,3)))
x_total = np.array(xList)

## Splitting the dataset into categories (wrong and correct)

In [0]:
# Get Indexes
y_wrong_index = np.where(y_total == 0)
y_correct_index = np.where(y_total == 1)
# Get Values
x_wrong = x_total[y_wrong_index]
y_wrong = y_total[y_wrong_index]
x_correct = x_total[y_correct_index]
y_correct = y_total[y_correct_index]


## Further splitting the frames into sequences

In [6]:
print("Initial shapes")
print(y_wrong.shape)
print(y_correct.shape)
print(x_wrong.shape)
print(x_correct.shape)

class FrameSet:

  def __init__(self, x, y):
    self.x = x
    self.y = y

def get_maximum_number_of_usable_frames(a):
  divider = nr_of_frames_per_sequence
  i = 1
  while (divider * i) < len(a):
    i = i + 1
  return divider * (i-1)

def transform_y_into_frame_sets(a):
  number_of_sets = get_maximum_number_of_usable_frames(a)
  print("Number of sets: {}".format(number_of_sets))
  sub_group_size = a[0:number_of_sets]
  return np.reshape(sub_group_size, (nr_of_frames_per_sequence, (int) (number_of_sets / nr_of_frames_per_sequence)))

def transform_x_into_frame_sets(a):
  number_of_sets = get_maximum_number_of_usable_frames(a)
  print("Number of sets: {}".format(number_of_sets))
  sub_group_size = a[0:number_of_sets]
  return np.reshape(sub_group_size, (nr_of_frames_per_sequence, (int) (number_of_sets / nr_of_frames_per_sequence), nr_of_keypoints, nr_of_values_per_keypoint))

#y_wrong = transform_y_into_frame_sets(y_wrong)
#y_correct = transform_y_into_frame_sets(y_correct)
#x_wrong = transform_x_into_frame_sets(x_wrong)
#x_correct = transform_x_into_frame_sets(x_correct)
print("After reshaping")
print(y_wrong.shape)
print(y_correct.shape)
print(x_wrong.shape)
print(x_correct.shape)

Initial shapes
(2317,)
(0,)
(2317, 25, 3)
(0, 25, 3)
After reshaping
(2317,)
(0,)
(2317, 25, 3)
(0, 25, 3)


## Recompine all wrong and correct arrays

In [7]:
# Only stack them when they are not empty
x_processed = None
y_processed = None
if np.any(x_correct != 0) and np.any(x_wrong != 0):
  x_processed = np.hstack((x_correct, x_wrong))
elif np.any(x_correct != 0):
  x_processed = x_correct
elif np.any(x_wrong != 0):
  x_processed = x_wrong
if np.any(y_correct != 0) and np.any(y_wrong != 0):
  y_processed = np.hstack((y_correct, y_wrong))
elif len(y_wrong) > 0:
  y_processed = y_wrong
elif len(y_correct) > 0:
  y_processed = y_correct
print(x_processed.shape)
print(y_processed)

(2317, 25, 3)
[0 0 0 ... 0 0 0]


## Finally, split them into trainint and testing datasets

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x_processed, y_processed, test_size=0.2)
print("Number of training Frame Sets {} and number of testing Frame sets {}".format(len(x_train), len(x_test)))

Number of training Frame Sets 1853 and number of testing Frame sets 464


## Defining the Model

In [19]:
print(x_train.shape)
model = keras.Sequential()
model.add(keras.layers.Dense(15, activation='relu', input_shape=(25,3)))
model.compile(optimizer='adam', loss='mean_squared_error')


(1853, 25, 3)


In [20]:
model.fit(x_train, y_train, epochs=30, batch_size=nr_of_frames_per_sequence)

ValueError: ignored